In [1]:
import json
from itertools import chain
from pprint import pprint
from time import time
import os

import numpy as np

from sklearn.metrics import accuracy_score

from gensim.models import Word2Vec
from gensim.corpora.dictionary import Dictionary


os.environ['THEANO_FLAGS'] = "device=gpu1"    
import theano
# theano.config.device = 'gpu' # Compute using GPU
# theano.config.floatX = 'float32'

from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Input
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.utils.visualize_util import plot

np.random.seed(1337)

print theano.config.device

Couldn't import dot_parser, loading of dot files will not be possible.


ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.
Using Theano backend.


gpu0


In [2]:
def indices_to_one_hot_encodings(index, vector_length):
    return [[1, 0] if i == index else [0, 1] for i in xrange(vector_length)]

In [3]:
# Load and process treebank data

treebank_file1 = open('json/OPTA-treebank-0.1.json')
treebank_file2 = open('skladnica_output.json')
treebank = chain(list(json.load(treebank_file1)), list(json.load(treebank_file2)))

X = []
y = []
for entry in treebank:
    tree = entry['parsedSent']
    words = []
    sentiment = None
    for index, node in enumerate(tree):
        word = node.split('\t')[1].lower()
        words.append(word)
        if node.split('\t')[10] == 'S':
            sentiment = index
    if sentiment:
        X.append(words)
        y.append(indices_to_one_hot_encodings(sentiment, len(words)))

dataset_length = len(X)
slicing_point = int(dataset_length*0.9)

X_train_raw = X[:slicing_point]
y_train_raw = y[:slicing_point]
X_test_raw = X[slicing_point+1:]
y_test_raw = y[slicing_point+1:]

treebank_vocabulary = set(chain(*X))
print len(treebank_vocabulary)

3906


In [4]:
print X_train_raw[2]
print y_train_raw[2]

[u'raczej', u'nie', u'dla', u'm\u0142odych', u'ch\u0142opc\xf3w', u'.']
[[0, 1], [0, 1], [0, 1], [1, 0], [0, 1], [0, 1]]


In [5]:
w2v_model = Word2Vec.load('w2v_allwiki_nkjp300_200.model')

In [6]:
# Import w2v's dictionary to a bag-of-words model
w2v_vocabulary = Dictionary()
w2v_vocabulary.doc2bow(w2v_model.vocab.keys(), allow_update=True)
print w2v_vocabulary.items()[:10]

[(0, u'zapachnie'), (1, u'PORADNI'), (2, u'Fitelberga'), (3, u'komedianta'), (4, u'Zaprzesta\u0107'), (5, u'Nampo'), (6, u'Schloendorff'), (7, u'zn\u0119kanym'), (8, u'synkopy'), (9, u'unifikacji')]


In [7]:
# Initialize dicts for representing w2v's dictionary as indices and 200-dim vectors
w2indx = {v: k+1 for k, v in w2v_vocabulary.items()}
w2vec = {word: w2v_model[word] for word in w2indx.keys()}

In [8]:
w2v_vocabulary_size = len(w2indx) + 1
w2v_vocabulary_dimension = len(w2vec.values()[0])

In [9]:
def map_treebank_words_to_w2v_indices(treebank_data, w2indx):
    treebank_data_vec = []
    for sentence in treebank_data:
        vectorized_sentence = []
        for word in sentence:
            try:
                vectorized_sentence.append(w2indx[word])
            except KeyError:  # words absent in w2v model will be indexed as 0s
                vectorized_sentence.append(0)
        treebank_data_vec.append(vectorized_sentence)
    return treebank_data_vec 

X_train = map_treebank_words_to_w2v_indices(X_train_raw, w2indx)
X_test = map_treebank_words_to_w2v_indices(X_test_raw, w2indx)

print X_test[4]

[51615, 277138, 416148, 422622, 318134, 584324, 176240, 503788, 0]


In [10]:
# Define numpy weights matrix for embedding layer
embedding_weights = np.zeros((w2v_vocabulary_size , w2v_vocabulary_dimension))
for word, index in w2indx.items():
    embedding_weights[index, :] = w2vec[word]

In [11]:
# max sentence length
max(
    len(max(X_train, key=lambda sentence: len(sentence))),
    len(max(X_test, key=lambda sentence: len(sentence)))
)

39

In [12]:
# Normalize sequences length to 40 (will be extended with 0s)
sentence_length = 40
X_train = sequence.pad_sequences(X_train, maxlen=sentence_length)
X_test = sequence.pad_sequences(X_test, maxlen=sentence_length)

y_train = sequence.pad_sequences(y_train_raw, maxlen=sentence_length, value=[0, 1])
y_test = sequence.pad_sequences(y_test_raw, maxlen=sentence_length, value=[0, 1])

print X_train[2]
print y_train[2]

[     0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0 580109 431241 193758 639684 453311      0]
[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]]


In [13]:
inputs = Input(shape=(sentence_length,), dtype='int32')

x = Embedding(
    input_dim=w2v_vocabulary_size, 
    output_dim=w2v_vocabulary_dimension,
    input_length=sentence_length,
    mask_zero=True,
    weights=[embedding_weights]
)(inputs)

lstm_out = LSTM(200, return_sequences=True)(x)

regularized_data = Dropout(0.3)(lstm_out)

predictions = TimeDistributed(Dense(2, activation='sigmoid'))(regularized_data)

model = Model(input=inputs, output=predictions)


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
model.summary()

# from IPython.display import SVG
# from keras.utils.visualize_util import model_to_dot

# SVG(model_to_dot(model).create(prog='dot', format='svg'))
# plot(model, to_file='model.png')

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
input_1 (InputLayer)               (None, 40)          0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)            (None, 40, 200)     141387200   input_1[0][0]                    
____________________________________________________________________________________________________
lstm_1 (LSTM)                      (None, 40, 200)     320800      embedding_1[0][0]                
____________________________________________________________________________________________________
dropout_1 (Dropout)                (None, 40, 200)     0           lstm_1[0][0]                     
___________________________________________________________________________________________

In [15]:
# theano.config.device = 'gpu'

In [16]:
batch_size = 50
n_epoch = 50


hist = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=n_epoch, 
                 validation_data=(X_test, y_test), verbose=2)

# epochs = 10

# for i in range(epochs):
#     print('Epoch', i, '/', epochs)
#     model.fit

Train on 1288 samples, validate on 143 samples
Epoch 1/50
44s - loss: 0.3807 - acc: 0.9374 - val_loss: 0.3047 - val_acc: 0.9717
Epoch 2/50
41s - loss: 0.2202 - acc: 0.9788 - val_loss: 0.2642 - val_acc: 0.9722
Epoch 3/50
49s - loss: 0.1542 - acc: 0.9825 - val_loss: 0.2567 - val_acc: 0.9706
Epoch 4/50
41s - loss: 0.1115 - acc: 0.9859 - val_loss: 0.2557 - val_acc: 0.9710
Epoch 5/50
43s - loss: 0.0820 - acc: 0.9882 - val_loss: 0.2557 - val_acc: 0.9705
Epoch 6/50
41s - loss: 0.0574 - acc: 0.9903 - val_loss: 0.2634 - val_acc: 0.9682
Epoch 7/50
39s - loss: 0.0399 - acc: 0.9912 - val_loss: 0.2757 - val_acc: 0.9680
Epoch 8/50
38s - loss: 0.0296 - acc: 0.9919 - val_loss: 0.2833 - val_acc: 0.9692
Epoch 9/50
39s - loss: 0.0232 - acc: 0.9919 - val_loss: 0.2933 - val_acc: 0.9682
Epoch 10/50
39s - loss: 0.0183 - acc: 0.9923 - val_loss: 0.3062 - val_acc: 0.9696
Epoch 11/50
39s - loss: 0.0172 - acc: 0.9919 - val_loss: 0.3138 - val_acc: 0.9694
Epoch 12/50
39s - loss: 0.0155 - acc: 0.9921 - val_loss: 0.3

In [17]:
predictions = model.predict(X_test, verbose=1)

143/143 [==============================] - 0s     


In [18]:
def change_encoding_word(word):
    return 1 if list(np.rint(word)) == [1, 0] else 0

def change_encoding(one_hot_encoded_sentence):
    # Switch from ndarray([[0.88, 0.11], [0.34, 0.98]]) encoding to [1, 0] encoding 
    # and finally index number
    normalized_sentence = []
    for word in one_hot_encoded_sentence:
        normalized_sentence.append(change_encoding_word(word))
    return normalized_sentence


total_accuracy = 0
for n, sentence in enumerate(predictions):
    index_of_sentiment = np.argmax(change_encoding(sentence))
    print change_encoding_word(y_test[n][index_of_sentiment])
    total_accuracy += change_encoding_word(y_test[n][index_of_sentiment])
    

print total_accuracy/float(len(y_test))
        


0
1
1
0
0
0
1
1
0
0
0
1
0
0
1
0
1
0
1
0
0
0
0
0
1
0
0
0
1
0
1
0
0
0
1
0
0
0
0
1
0
0
0
1
1
0
0
1
0
0
1
1
0
0
0
1
0
1
0
0
0
1
1
0
1
0
0
0
1
0
1
0
1
0
0
0
0
1
1
1
1
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
1
0
1
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0.314685314685
